In [23]:
import folium

# 示例轨迹数据（你可换成火车或航班的经纬度点）
track_points = [
    [31.2304, 121.4737],  # 上海
    [32.0603, 118.7969],  # 南京
    [34.3416, 108.9398],  # 西安
    [39.9042, 116.4074]   # 北京
]

# 创建基础地图
m = folium.Map(location=[34, 113], zoom_start=5, tiles="CartoDB positron")

# 在地图上加入起点终点标记
folium.Marker(track_points[0], popup="起点", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(track_points[-1], popup="终点", icon=folium.Icon(color="red")).add_to(m)

# 添加一个空的 polyline（JS 后面动态填充）
#folium.PolyLine([], color='deepskyblue', weight=4, opacity=0.8, tooltip="轨迹线").add_to(m)

# 定义 JavaScript 动画脚本
js_code = f"""
<script>
var points = {track_points};  // Python轨迹点传给JS
var map = Object.values(window).find(v => v instanceof L.Map); // 获取folium的地图对象
var polyline = L.polyline([], {{color: 'deepskyblue', weight: 4, opacity: 0.8}}).addTo(map);

let i = 0;
function drawNext() {{
    if (i < points.length) {{
        polyline.addLatLng(points[i]);
        map.panTo(points[i], {{animate: true, duration: 1}});
        i++;
        setTimeout(drawNext, 1000); // 每隔1秒画一个点，可调节速度
    }}
}}
drawNext();
</script>
"""

# 保存HTML后附加JS动画
m.save("trajectory_grow.html")

with open("trajectory_grow.html", "a", encoding="utf-8") as f:
    f.write(js_code)

print("✅ 轨迹逐渐显现动画已保存为 trajectory_grow.html")


✅ 轨迹逐渐显现动画已保存为 trajectory_grow.html


In [56]:
import numpy as np



def interpolate_line(start, end, num_points=50):
    """线性插值——生成从start到end的num_points个经纬度点"""
    lons = np.linspace(start[1], end[1], num_points)
    lats = np.linspace(start[0], end[0], num_points)
    track = []
    for i,j in zip(lats,lons):
        track.append([i,j])
    return track

from geopy.distance import great_circle
import numpy as np

def interpolate_arc(start, end, num_points=50, deviation=0.5):
    """
    弧线插值——模拟飞行轨迹。
    deviation: 弧线偏移强度，0表示直线，值越大弯曲越明显
    """
    
    
    # 经纬度转弧度
    lat1, lon1 = np.radians(start)
    lat2, lon2 = np.radians(end)

    # 插值角度
    f = np.linspace(0, 1, num_points)
    d = 2 * np.arcsin(np.sqrt(
        np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2 - lon1)/2)**2
    ))

    A = np.sin((1 - f) * d) / np.sin(d)
    B = np.sin(f * d) / np.sin(d)

    x = A * np.cos(lat1) * np.cos(lon1) + B * np.cos(lat2) * np.cos(lon2)
    y = A * np.cos(lat1) * np.sin(lon1) + B * np.cos(lat2) * np.sin(lon2)
    z = A * np.sin(lat1) + B * np.sin(lat2)

    # 添加弧线偏移
    x += deviation * np.random.uniform(-0.001, 0.001)
    y += deviation * np.random.uniform(-0.001, 0.001)

    lat = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
    lon = np.degrees(np.arctan2(y, x))
    track = []
    for i,j in zip(lat,lon):
        if j<0:
            j+=360
        track.append([i,j])
    return track

import numpy as np

def interpolate_flight_arc_adaptive(start, end, num_points=100, base_height=0.08):
    """
    生成自适应弧度的飞行轨迹（球面插值）
    - start, end: (lat, lon)
    - base_height: 弧度基准（越大弧线越高）
    """
    lat1, lon1 = np.radians(start)
    lat2, lon2 = np.radians(end)

    # 球面距离 (弧度)
    delta = np.arccos(
        np.sin(lat1) * np.sin(lat2) +
        np.cos(lat1) * np.cos(lat2) * np.cos(lon2 - lon1)
    )

    # 根据距离动态调整弧度高度（0.02 ~ base_height）
    # 这里假设 delta 在 0~π 之间，delta越大，arc越小
    arc_height = base_height * np.exp(-delta * 5)
    arc_height = max(arc_height, 0.015)  # 保证最小弧度

    result = []
    for f in np.linspace(0, 1, num_points):
        A = np.sin((1 - f) * delta) / np.sin(delta)
        B = np.sin(f * delta) / np.sin(delta)
        x = A * np.cos(lat1) * np.cos(lon1) + B * np.cos(lat2) * np.cos(lon2)
        y = A * np.cos(lat1) * np.sin(lon1) + B * np.cos(lat2) * np.sin(lon2)
        z = A * np.sin(lat1) + B * np.sin(lat2)

        lat = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
        lon = np.degrees(np.arctan2(y, x))

        # 添加弧度偏移
        lat += arc_height * np.sin(f * np.pi)
        
        if lon<0:
            lon+=360
        result.append([lat, lon])

    return result


import folium

# 示例数据
train_start = [31.2304, 121.4737]  # 上海
train_end = [34.3416, 108.9398]    # 西安
plane_start = [31.2304, 360-121.4737]  # 上海
plane_end = [39.9042, 116.4074]    # 北京

# 轨迹插值
train_track = interpolate_line(train_start, train_end, num_points=100)
plane_track = interpolate_flight_arc_adaptive(plane_start, plane_end, num_points=100)

# 创建地图
m = folium.Map(location=[34, 113], zoom_start=5, tiles="CartoDB positron")

# 添加初始空轨迹
#folium.PolyLine([], color='deepskyblue', weight=4, tooltip="火车轨迹").add_to(m)
#folium.PolyLine([], color='crimson', weight=4, tooltip="飞行轨迹").add_to(m)

# 添加起点终点标记
folium.Marker(train_start, popup="火车起点", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(train_end, popup="火车终点", icon=folium.Icon(color="red")).add_to(m)
folium.Marker(plane_start, popup="航班起点", icon=folium.Icon(color="blue")).add_to(m)
folium.Marker(plane_end, popup="航班终点", icon=folium.Icon(color="orange")).add_to(m)

# 动态绘制 JS 脚本
js_code = f"""
<script>
var points = {train_track+plane_track};  // Python轨迹点传给JS
var map = Object.values(window).find(v => v instanceof L.Map); // 获取folium的地图对象
var polyline = L.polyline([], {{color: 'deepskyblue', weight: 4, opacity: 0.8}}).addTo(map);

let i = 0;
function drawNext() {{
    if (i < points.length) {{
        polyline.addLatLng(points[i]);
        map.panTo(points[i], {{animate: true, duration: 1}});
        i++;
        setTimeout(drawNext, 50); // 每隔1秒画一个点，可调节速度
    }}
}}
drawNext();
</script>
"""

js_code = f"""
<script>
var map = Object.values(window).find(v => v instanceof L.Map); // 获取folium的地图对象
var polyline = L.polyline([], {{color: 'deepskyblue', weight: 4, opacity: 0.8}}).addTo(map);
var iconEmoji = "🚆"
var train = "🚆"
var plane = "✈️"
function animateMarker(path, duration) {{
  let marker = L.marker(path[0], {{
    icon: L.divIcon({{
      className: 'animated-marker',
      html: '<div style="font-size:24px;">' + iconEmoji + '</div>'
    }})
  }}).addTo(map);

  let i = 0;
  let steps = path.length;
  function move() {{
    if (i == 60) {{
        map.removeLayer(marker);
        marker = L.marker(path[i], {{
            icon: L.divIcon({{
                className: 'animated-marker',
                html: '<div style="font-size:24px;">' + plane + '</div>'
                }})
      }}).addTo(map);
    }}
    if (i < steps) {{
      polyline.addLatLng(path[i]);
      map.panTo(path[i], {{animate: true, duration: 1}});
      marker.setLatLng(path[i]);
      i++;
      setTimeout(move, duration / steps);
    }} else {{
      map.removeLayer(marker);
    }}
  }}
  move();
}}

var train_path = {train_track+plane_track};
var flight_path = {plane_track};

animateMarker(train_path, 6000);
// animateMarker(flight_path, 4000);
</script>
"""




# 保存 HTML 并附加 JS 动画
m.save("trajectory_interpolated.html")
with open("trajectory_interpolated.html", "a", encoding="utf-8") as f:
    f.write(js_code)

print("✅ 轨迹插值+动态显现地图已保存为 trajectory_interpolated.html")


✅ 轨迹插值+动态显现地图已保存为 trajectory_interpolated.html
